<a href="https://colab.research.google.com/github/swilli21/Agentic-AI/blob/master/PromptTemplates_SequentialChains_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#  Install required packages
!pip install -qU langchain langchain-google-genai google-generativeai==0.8.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.1/500.1 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 10.5 MB/s eta 0:00:00


In [3]:
import os, sys, platform
from google.colab import userdata
from datetime import datetime

# LangChain + Gemini
import langchain, langchain_core
from langchain_google_genai import ChatGoogleGenerativeAI       # to work with Gemini models
from langchain_core.prompts import PromptTemplate               # to format prompts
from langchain_core.output_parsers import StrOutputParser       # to extract plain text from responses
from langchain_core.runnables import RunnablePassthrough        # to chain steps

In [6]:
#  Set your Gemini API key
os.environ["GOOGLE_API_KEY"] = userdata.get("GEMINI_API_KEY_DEFAULT")

Temperature-  (typically 0 to 2+) that controls the randomness and creativity of generated text by scaling the probability distribution of the next token

In [7]:
# Recommended for demos: fast & economical
model_name = "gemini-2.5-flash"

llm = ChatGoogleGenerativeAI(
    model=model_name,
    convert_system_message_to_human=True,  # smoother behavior with system prompts
    temperature=0.7, # higher the temperature (0 to 1) allows the LLM to be creative with its reponse
)
print(f" Ready with model: {model_name}")

 Ready with model: gemini-2.5-flash


# Simple sequential chain (Ideas → Summary)

In [8]:
# Step 1: generate ideas
idea_prompt = PromptTemplate.from_template(
    "Generate 3 creative ideas about: {topic}. Return them as a numbered list."
)

# Step 2: summarize the ideas
summary_prompt = PromptTemplate.from_template(
    "Summarize the ideas below in 2 friendly sentences for a beginner:\n\n{ideas}"
)


In [9]:
# Chain

idea_chain = idea_prompt | llm | StrOutputParser()

### Sequential Chain without memory

- Generate the key ideas
- The key topic is passed directly to the summary prompt
- Then to the LLM
- Finally to the output

In [10]:
# Compose a chain where the output of idea_chain is fed into the summary prompt
sequential_chain = {
    "ideas": idea_chain,
    "topic": RunnablePassthrough()
} | summary_prompt | llm | StrOutputParser()

In [11]:
topic = "using AI to improve school homework feedback"
Creative_ideas = idea_chain.invoke({"topic": topic}) # first chain generate the topics
summary = sequential_chain.invoke({"topic": topic}) # second chain passes the topics to be summarized

In [12]:
print(" Topic:", topic)
print(" Creative Ideas : ", Creative_ideas)
print(" Summary:\n", summary)

 Topic: using AI to improve school homework feedback
 Creative Ideas :  Here are 3 creative ideas about using AI to improve school homework feedback:

1.  **The AI "Conceptual Coach" with Interactive Remediation:**
    Instead of just marking an answer wrong, an AI would act as a personalized conceptual coach. When a student submits an incorrect or incomplete answer, the AI wouldn't just give the right answer. Instead, it would engage in a **guided dialogue**, asking probing questions to help the student identify their own misunderstanding, provide scaffolded hints (e.g., "Think about the main principle of X," or "What was the first step we learned for solving Y?"), and then offer alternative examples or analogies to clarify the concept. If the student is still struggling, the AI could generate a few **micro-practice problems** specifically tailored to the identified gap in their understanding, ensuring they master the prerequisite concept before moving on. This transforms static feedb